In [59]:
!pip install plotly
!pip install dtreeviz

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#Libs para importar os modelos que serão utilizados e para verificar a precisão dos mesmo
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC

#Lib criada pela equipe para fazer os devidos tratamentos dos tweets.
from remove_normalizer import ajustandoDados

#lib de plotagem
import plotly.graph_objects as go

#Lib para salvarmos os modelos, CountVectorizer e as categorias alvos para utilizarmos posteriormente sem a necessidade de treinarmos o modelo novamente
import joblib

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rafae\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\rafae\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\rafae\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\rafae\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Lemos o aqruivo excel e transformamos em um dataframe

In [2]:
tweets = pd.read_excel('C:/Users/rafae/OneDrive/Documents/GitHub/Tweet_Classifier/Final/dados/train_data/dados_unidos_oficial_semDuplicatas.xlsx')
display(tweets.head())

,full_text,classificacao_texto,classificacao
0,acho q o bozo errou o time chegou meio atrasado,neutro,0
1,jair em desespero,negativo,-1
2,armacao do jair,negativo,-1
3,camilamoreno13_ jonvlogs raciocinio bom e o t...,negativo,-1
4,campanha de bolsonaro acusacao sem provas reai...,negativo,-1


Retirando colunas que não serão utilizadas pelo modelo

Selecionamos os resultados que o modelo que iremos utilizar deverá dar.

In [3]:
categoria_alvo = tweets['classificacao_texto'].unique()

print(categoria_alvo)


['neutro' 'negativo' 'positivo']


#### Verificando a quantidade de dados por classificação

Aqui estamos fazendo um groupby da coluna "classificacao_texto", onde fala se o tweet é positivo, negativo ou neutro. Com isso

In [4]:
classificacao = tweets.groupby('classificacao_texto').classificacao.value_counts()
classificacao = pd.DataFrame(classificacao)
classificacao.reset_index(names=['classificacao_texto','classificacao1'],inplace=True)
classificacao.rename(columns = {'classificacao1':'classificacao', 'classificacao':'soma'}, inplace = True)
display(classificacao.head())
print(f'Total de Classificação de Lula: {classificacao.soma.sum()}')

,classificacao_texto,classificacao,soma
0,negativo,-1,200
1,neutro,0,79
2,positivo,1,134


Total de Classificação de Lula: 413


Aqui verificamos que dentre 413 tweets as classificações ficam distribuidas da seguinte maneira, 48% classificações negativas, 19% neutras e 32% positivas.

In [5]:
fig = go.Figure(data=[
    go.Bar(name='Total', x=classificacao['classificacao_texto'], y=classificacao["soma"], text=classificacao["soma"])
])
# Change the bar mode
fig.update_layout(title = "Quantidade de Classificação")
fig.show()

Aqui criamos uma máscara para selecionar os tweets apenas para um candidato.

Essas máscaras foram criadas para verificarmos como estão distribuidas as classificações por candidato.

In [6]:
#Dataframe que aparece palavras referentes ao Lula
searchLula = ['lula' , 'lulalivre' , 'lulaladrao' , 'molusco' , 'lulaladrão' , 'lulapresidente','lula13']
mask_lula = tweets['full_text'].str.contains('|'.join(searchLula))
lula_df = tweets[mask_lula]
#Dataframe que aparece palavras referentes ao bolsonaro
searchBolsonaro = ['bolsonaro', 'bozo', 'bozonaro', 'elenão', 'elesim', 'jair', 'mito',' bolsonaro22']
mask_bolso = tweets['full_text'].str.contains('|'.join(searchBolsonaro))
bolsonaro_df = tweets[mask_bolso]

Aqui geramos o dataframe apenas com os tweets que mencionam o Lula.

In [7]:
classificacao_lula = lula_df.groupby('classificacao_texto').classificacao.value_counts()
classificacao_lula = pd.DataFrame(classificacao_lula)
classificacao_lula.reset_index(names=['classificacao_texto','classificacao1'],inplace=True)
classificacao_lula.rename(columns = {'classificacao1':'classificacao', 'classificacao':'soma'}, inplace = True)
display(classificacao_lula.head())
print(f'Total de Classificação de Lula: {classificacao_lula.soma.sum()}')

,classificacao_texto,classificacao,soma
0,negativo,-1,102
1,neutro,0,25
2,positivo,1,109


Total de Classificação de Lula: 236


Aqui geramos o dataframe apenas com os tweets que mencionam o Bolsonaro.

In [8]:
classificacao_bolsonaro = bolsonaro_df.groupby('classificacao_texto').classificacao.value_counts()
classificacao_bolsonaro = pd.DataFrame(classificacao_bolsonaro)
classificacao_bolsonaro.reset_index(names=['classificacao_texto','classificacao1'],inplace=True)
classificacao_bolsonaro.rename(columns = {'classificacao1':'classificacao', 'classificacao':'soma'}, inplace = True)
display(classificacao_bolsonaro.head())
print(f'Total de Classificação de Bolsonaro: {classificacao_bolsonaro.soma.sum()}')

,classificacao_texto,classificacao,soma
0,negativo,-1,125
1,neutro,0,28
2,positivo,1,60


Total de Classificação de Bolsonaro: 213


Quando olhamos para classificação por presidente veficamos que Lula que possuí 236 tweets que é mencionado e a classificação desses tweets são distribuidas da seguinte maneira, como mostrada na fig. X, 43% são classificações negativas, 11% são neutras e 46% são possitivas. Quando olhamos para a distribuição de Bolsonaro, onde seu nome é mencionado em 213 tweets percebemos que a distribuição é dada da seguinte maneira: 59% são negativas, 13% são neutras e 28% são positivas.

In [9]:
fig = go.Figure(data=[
    go.Bar(name='Lula', x=classificacao_lula['classificacao_texto'], y=classificacao_lula["soma"], text=classificacao_lula["soma"]),
    go.Bar(name='Bolsonaro', x=classificacao_bolsonaro['classificacao_texto'], y=classificacao_bolsonaro["soma"], text=classificacao_bolsonaro["soma"]),
])
# Change the bar mode
fig.update_layout(barmode='group', title = 'Comparação de Quantidade de Classificações entre Lula e Bolsonaro')
fig.show()

Lembrando nem todos os tweets mencionam os presidentes diretamente e tweets podem mencionar os dois candidatos, na época que foram baixados os dados. Então podemos verificar também que os dois candidatos possuem a quantidade de dados equilibradas no total.

#### Tratamento

Aqui fazemos os tratamentos dos dados removendo as stopwords.

In [10]:
tweets['full_text'] = tweets['full_text'].apply(ajustandoDados.remove_stopwords)
tweets['full_text'][1]

['jair', 'desespero']

Aqui é normalizada as palavras do dataset utilizando lemmatize, aqui encontrando-se o lema da palavra de maneira que é considerado o contexto da frase.

[Conceito de lemmatize](https://www.guru99.com/stemming-lemmatization-python-nltk.html)

[Conceito de lemma ou lema](https://simple.wikipedia.org/wiki/Lemma_(linguistics))

In [11]:
tweets['full_text'] = tweets['full_text'].apply(ajustandoDados.lemmatize_word)
tweets['full_text'][1]

'jair desespero'

#### Criando o modelo

Primeiro iremos selecionar os dados que serão previstos e o que serão utilizados para prever.

Após isso imprimos o formato de x e y.

In [27]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
#Transformamos os tweets em array.
x = np.array(tweets.iloc[:,0].values)
#Transformamos as classificações em array.
y = np.array(tweets['classificacao'].values)

#Aqui no CountVectorizer definifimos para ignorar termos com uma frequência menor que 0.3 e maior que 0.7 nos documentos.
#Após isso definimos as a matriz de características a partir de 200 palavras mais frequentes nos documentos.
cv = CountVectorizer(min_df=5, max_features=200)
x_cv = cv.fit_transform(tweets['full_text']).toarray()

#Aqui no TfidfVectorizer definifimos para ignorar termos com uma frequência menor que 0.3 e maior que 0.7 nos documentos.
#Após isso definimos as a matriz de características a partir de 200 palavras mais frequentes nos documentos.
tfidf = TfidfVectorizer(min_df=5, max_features=200)
x_tfidf = tfidf.fit_transform(tweets['full_text']).toarray()

#Aqui verificamos o shape de X que são os tweets, tanto para tfidf quanto para cv, e de y que são a categoria alvo.
print("X_tfidf.shape = ",x_tfidf.shape)
print("X_cv .shape = ",x_cv.shape)
print("y.shape = ",y.shape)

X_tfidf.shape =  (413, 175)
X_cv .shape =  (413, 175)
y.shape =  (413,)


Aqui iremos separar os dados entre dados de treino e dados de teste.

In [14]:
from sklearn.model_selection import train_test_split
#Aqui divimos o dataset em teste e treino, 80% do dataset será para treino do modelo e 20% para teste. Utilizando os dados do tfidf
x_train_tfidf, x_test_tfidf, y_train_tfidf, y_test_tfidf = train_test_split(x_tfidf, y, test_size = 0.20, random_state = 0, shuffle = True)
print(len(x_train_tfidf))
print(len(x_test_tfidf))

330
83


In [15]:
#Aqui divimos o dataset em teste e treino, 80% do dataset será para treino do modelo e 20% para teste. Utilizando os dados do CountVectorizer
x_train_cv, x_test_cv, y_train_cv, y_test_cv = train_test_split(x_cv, y, test_size = 0.20, random_state = 0, shuffle = True)
print(len(x_train_cv))
print(len(x_test_cv))

330
83


Aqui definimos vários modelos para prevermos qual obtem o melhor resultado, isso poderemos ver na tabela que é criada com o resultado com o score de cada modelo.

In [17]:
def run_model_tfidf(model_name, x_train, y_train):
    '''
      Aqui definimos uma função que irá rodar os modelos de Decision Tree, Random Forests e SVC, utilizando os dados que foram utilizados o TfdifVectorizer
    '''
    mdl=''
    if model_name == 'Random Forest':
      #Definimos que a floresta terá 50 árvores.
      mdl = RandomForestClassifier(n_estimators=50)
    elif model_name == 'Decision Tree Classifier':
      mdl = DecisionTreeClassifier()
    elif model_name == 'Support Vector Classifier':
      mdl = SVC()

    #Fazemos com que sejam construidas as florestas, no caso do modelo de Random Forests,
    #ou que sejam construída a árvore, no caso da Decision Tree.
    mdl.fit(x_train_tfidf, y_train_tfidf)
    
    
    #Aqui testamos os modelo treinado nos dados de teste
    y_pred_tfidf = mdl.predict(x_test_tfidf)
    
    # Performance metrics
    accuracy = round(accuracy_score(y_test_tfidf, y_pred_tfidf) * 100, 2)

    # Get precision, recall, f1 scores
    precision_macro, recall_macro, f1score_macro, support_macro = score(y_test_tfidf, y_pred_tfidf, average='macro')
    
    # Add performance parameters to list
    perform_list_tfidf.append(dict([
      ('Model', model_name),
      ('Test Accuracy_tfidf', round(accuracy, 2)),
      ('F1_macro_tfidf', round(f1score_macro, 2))
    ]))


In [18]:
def run_model_cv(model_name, x_train, y_train):
    '''
      Aqui definimos uma função que irá rodar os modelos de Decision Tree, Random Forests e SVC, utilizando os dados que foram utilizados o CountVectorizer
    '''
    mdl=''
    if model_name == 'Random Forest':
      #Definimos que a floresta terá 50 árvores.
      mdl = RandomForestClassifier(n_estimators=50)
    elif model_name == 'Decision Tree Classifier':
      mdl = DecisionTreeClassifier()
    elif model_name == 'Support Vector Classifier':
      mdl = SVC()

    #Fazemos com que sejam construidas as florestas, no caso do modelo de Random Forests,
    #ou que sejam construída a árvore, no caso da Decision Tree.
    mdl.fit(x_train_cv, y_train_cv)
    
    
    #Aqui testamos os modelo treinado nos dados de teste
    y_pred_cv = mdl.predict(x_test_cv)
    
    # Performance metrics
    accuracy = round(accuracy_score(y_test_cv, y_pred_cv) * 100, 2)

    # Get precision, recall, f1 scores
    precision_macro, recall_macro, f1score_macro, support_macro = score(y_test_cv, y_pred_cv, average='macro')
    
    # Add performance parameters to list
    perform_list_cv.append(dict([
      ('Model', model_name),
      ('Test Accuracy_cv', round(accuracy, 2)),
      ('F1_macro_cv', round(f1score_macro, 2))
    ]))


Aqui vemos o resultado de quando utilizamos o modelo nos dados de teste.

In [19]:
#Criando a lista de performance para os modelos que utilizaram TfidfVectorizer
perform_list_tfidf = []

#Rodando o modelo e verificando 
run_model_tfidf('Random Forest', x_train_tfidf, y_train_tfidf)
run_model_tfidf('Decision Tree Classifier', x_train_tfidf, y_train_tfidf)
run_model_tfidf('Support Vector Classifier', x_train_tfidf, y_train_tfidf)

result_tfidf = pd.DataFrame(perform_list_tfidf)
display(result_tfidf)

,Model,Test Accuracy_tfidf,F1_macro_tfidf
0,Random Forest,68.67,0.66
1,Decision Tree Classifier,61.45,0.61
2,Support Vector Classifier,67.47,0.62


In [20]:
#Criando a lista de performance que utilizaram CountVectorizer
perform_list_cv = []

#Rodando o modelo e verificando 
run_model_cv('Random Forest', x_train_cv, y_train_cv)
run_model_cv('Decision Tree Classifier', x_train_cv, y_train_cv)
run_model_cv('Support Vector Classifier', x_train_cv, y_train_cv)

result_cv = pd.DataFrame(perform_list_cv)
display(result_cv)

,Model,Test Accuracy_cv,F1_macro_cv
0,Random Forest,60.24,0.59
1,Decision Tree Classifier,56.63,0.54
2,Support Vector Classifier,68.67,0.67


#### Validando o Modelo

Aqui criamos um dataframe com frases novas para verificarmos como o modelo se comporta com frases novas.

In [22]:
teste = [['Gosto muito do presidente Lula'],
         ['Gosto muito do presidente Bolsonaro'], 
         ['Lula foi um ladrão e merece ir para a cadeia'],
         ['Bolsonaro foi um ladrão e merece ir para a cadeia'],
         ['Bolsonaro foi um dos piores presidentes do brasil, robou e fez o que queria para se manter no poder'],
         ['Lula foi um dos piores presidentes do brasil, robou e fez o que queria para se manter no poder'], 
         ['jair bolsonaro foi muito bom para o povo brasileiro, deu oportunidades, emprego'],
         ['lula foi muito bom para o povo brasileiro, deu oportunidades, emprego']]
dfTeste = pd.DataFrame(teste, columns=['frase_teste'])
ajustandoDados.remove_misc1(dfTeste, 'frase_teste')
dfTeste['frase_teste'] = dfTeste['frase_teste'].apply(ajustandoDados.remove_stopwords)
dfTeste['frase_teste'] = dfTeste['frase_teste'].apply(ajustandoDados.lemmatize_word)
dfTeste

,frase_teste
0,gosto presidente lula
1,gosto presidente bolsonaro
2,lula ladrao merece ir cadeia
3,bolsonaro ladrao merece ir cadeia
4,"bolsonaro piores presidentes brasil , robou fe..."
5,"lula piores presidentes brasil , robou fez que..."
6,"jair bolsonaro bom povo brasileiro , deu oport..."
7,"lula bom povo brasileiro , deu oportunidades ,..."


Aqui pegamos os mesmos CountVectorizer, as mesmas keywords arguments para os modelos e as mesma categorias alvo.

In [23]:
#Criando Dataframe para salvar os resultados dos modelos
resultados = pd.DataFrame()
resultados['frases'] = dfTeste['frase_teste']

In [24]:
#Aqui iremos utilizar os modelos que utilizaram TfidfVectorizer em daods inéditos, que não foram utilizados
y_teste_tfidf= tfidf.transform(dfTeste['frase_teste']).toarray()
mdl_RF_tfidf = RandomForestClassifier(n_estimators=50).fit(x_train_tfidf, y_train_tfidf)
mdl_DT_tfidf = DecisionTreeClassifier().fit(x_train_tfidf, y_train_tfidf)
mdl_SVC_tfidf = OneVsRestClassifier(SVC()).fit(x_train_tfidf, y_train_tfidf)


y_pred_RF_tfidf = mdl_RF_tfidf.predict(y_teste_tfidf)
y_pred_DT_tfidf = mdl_DT_tfidf.predict(y_teste_tfidf)
y_pred_SVC_tfidf = mdl_SVC_tfidf.predict(y_teste_tfidf)

resultado_RF_tfidf = categoria_alvo[y_pred_RF_tfidf]
resultado_DT_tfidf = categoria_alvo[y_pred_DT_tfidf]
resultado_SVC_tfidf = categoria_alvo[y_pred_SVC_tfidf]


#Aqui iremos utilizar os modelos que utilizaram TfidfVectorizer em daods inéditos, que não foram utilizados
y_teste_cv = cv.transform(dfTeste['frase_teste']).toarray()
mdl_RF_cv = RandomForestClassifier(n_estimators=50).fit(x_train_cv, y_train_cv)
mdl_DT_cv = DecisionTreeClassifier().fit(x_train_cv, y_train_cv)
mdl_SVC_cv = OneVsRestClassifier(SVC()).fit(x_train_cv, y_train_cv)


y_pred_RF_cv = mdl_RF_cv.predict(y_teste_cv)
y_pred_DT_cv = mdl_DT_cv.predict(y_teste_cv)
y_pred_SVC_cv = mdl_SVC_cv.predict(y_teste_cv)

resultado_RF_cv = categoria_alvo[y_pred_RF_cv]
resultado_DT_cv = categoria_alvo[y_pred_DT_cv]
resultado_SVC_cv = categoria_alvo[y_pred_SVC_cv]

In [25]:
#Colocando os resultados dos modelos 
resultados['RF_tfidf'] = resultado_RF_tfidf
resultados['RF_cv'] = resultado_RF_cv
resultados['DT_tfidf'] = resultado_DT_tfidf
resultados['DT_cv'] = resultado_DT_cv
resultados['SVC_tfidf'] = resultado_SVC_tfidf
resultados['SVC_cv'] = resultado_SVC_cv

display(resultados)

,frases,RF_tfidf,RF_cv,DT_tfidf,DT_cv,SVC_tfidf,SVC_cv
0,gosto presidente lula,negativo,negativo,negativo,negativo,negativo,negativo
1,gosto presidente bolsonaro,negativo,negativo,neutro,negativo,negativo,positivo
2,lula ladrao merece ir cadeia,positivo,positivo,negativo,positivo,positivo,positivo
3,bolsonaro ladrao merece ir cadeia,positivo,positivo,positivo,positivo,positivo,positivo
4,"bolsonaro piores presidentes brasil , robou fe...",positivo,positivo,positivo,positivo,positivo,positivo
5,"lula piores presidentes brasil , robou fez que...",positivo,positivo,negativo,positivo,positivo,negativo
6,"jair bolsonaro bom povo brasileiro , deu oport...",positivo,positivo,positivo,positivo,positivo,positivo
7,"lula bom povo brasileiro , deu oportunidades ,...",negativo,neutro,negativo,neutro,positivo,negativo


No dataframe nos mostra que os resultados dos modelos e sua respectiva representação numérica/vectorial do texto, TfidfVectorizer e CountVectorizer, percebemos que os resultados tendem a classificar as frases com 'bolsonaro' nos tweets como positivas e que contêm 'lula' como negativas e neutras. Notamos isto tanto nos diferentes modelos, isto pode ter como a quantidade de dados é baixa e alguns tweets usados mencionam ambos os candidatos na mesma frase, notamos isto quando verificamos a figura onde comparamos a quantidade de classificação por menção do candidato, onde notamos que ela está acima da quantidade de dados presentes no estudo.

Agora iremos salvar os modelos já treinados, os vetorizadores que são o CountVectorizer e o TfidfVectorizer e as categorias alvo para utilizarmos posteriormente sem a necessidade de treina-los novamente.

In [28]:
#Salva o modelo de random forest
filename = 'C:/Users/rafae/OneDrive/Documents/GitHub/Tweet_Classifier/Final/cod/modelo/classificadores/RandomForestClassifier_tweets.sav'
model = RandomForestClassifier(n_estimators=50).fit(x_train_tfidf, y_train_tfidf)
joblib.dump(model, filename)

#Salva o modelo de Decision Tree
filename = 'C:/Users/rafae/OneDrive/Documents/GitHub/Tweet_Classifier/Final/cod/modelo/classificadores/DecisionTreeClassifier_tweets.sav'
model = DecisionTreeClassifier().fit(x_train_tfidf, y_train_tfidf)
joblib.dump(model, filename)

#Salva o modelo de SVC
filename = 'C:/Users/rafae/OneDrive/Documents/GitHub/Tweet_Classifier/Final/cod/modelo/classificadores/SVC_tweets.sav'
model = OneVsRestClassifier(SVC()).fit(x_train_tfidf, y_train_tfidf)
joblib.dump(model, filename)

#Salva os vetorizadores:
filename = 'C:/Users/rafae/OneDrive/Documents/GitHub/Tweet_Classifier/Final/cod/modelo/classificadores/CV_tweets.sav'
joblib.dump(cv, filename)

filename = 'C:/Users/rafae/OneDrive/Documents/GitHub/Tweet_Classifier/Final/cod/modelo/classificadores/TFIDF_tweets.sav'
joblib.dump(tfidf, filename)

#Salva as categorias
filename = 'C:/Users/rafae/OneDrive/Documents/GitHub/Tweet_Classifier/Final/cod/modelo/classificadores/categoria_alvo_tweets.sav'
joblib.dump(categoria_alvo, filename)

['C:/Users/rafae/OneDrive/Documents/GitHub/Tweet_Classifier/Final/cod/modelo/classificadores/categoria_alvo_tweets.sav']